In [1]:
import tensorflow as tf
import pandas as pd
import os
import json
import shutil
from tensorflow.python.lib.io.file_io import FileIO as open_file

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
DATASET_CSV="gs://ml-research-injenia/estimators/datasets/superhero-set/preproc_trainset.csv"
EVALSET_CSV="gs://ml-research-injenia/estimators/datasets/superhero-set/preproc_evalset.csv"
MODEL_DIR="gs://ml-research-injenia/estimators/trainings-superheroes-v2/linear-classifier/test06"

BATCH_SIZE = 100
TRAIN_STEPS= 100000
LEARNING_RATE=0.01

In [3]:
with open_file(DATASET_CSV, "r") as f:
    df = pd.read_csv(f)
for c in df.columns:
    if df[c].dtypes.name == 'bool':
        df[c]=df[c].map({True: 'TRUE', False: 'FALSE'})
df

,index,Gender,Eye_color,Race,Hair_color,Height,Publisher,Skin_color,Alignment,Weight,...,Web_Creation,Reality_Warping,Odin_Force,Symbiote_Costume,Speed_Force,Phoenix_Force,Molecular_Dissipation,Vision_Cryo,Omnipresent,Omniscient
0,Molten Man,Male,gold,-,Gold,196.0,Marvel Comics,-,bad,248.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
1,Killer Frost,Female,blue,Human,Blond,-99.0,DC Comics,blue,bad,-99.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
2,Raven,Female,indigo,Human,Black,165.0,DC Comics,-,neutral,50.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
3,Vegeta,Male,-,Saiyan,Black,168.0,Shueisha,-,bad,73.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
4,Gog,Male,-,-,-,-99.0,DC Comics,-,bad,-99.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
5,Redeemer II,Male,-,-,-,-99.0,Image Comics,-,bad,-99.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
6,Captain Marvel II,Male,blue,Human,Black,175.0,DC Comics,-,good,74.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
7,Proto-Goblin,Male,green,-,Blond,-99.0,Marvel Comics,-,bad,-99.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
8,Sentry,Male,blue,Mutant,Blond,188.0,Marvel Comics,-,neutral,87.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
9,Kylo Ren,Male,-,Human,-,-99.0,George Lucas,-,bad,-99.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [4]:
with open_file(EVALSET_CSV, "r") as f:
    df_eval = pd.read_csv(f)
for c in df_eval.columns:
    if df_eval[c].dtypes.name == 'bool':
        df_eval[c]=df_eval[c].map({True: 'TRUE', False: 'FALSE'})
df_eval

,index,Gender,Eye_color,Race,Hair_color,Height,Publisher,Skin_color,Alignment,Weight,...,Web_Creation,Reality_Warping,Odin_Force,Symbiote_Costume,Speed_Force,Phoenix_Force,Molecular_Dissipation,Vision_Cryo,Omnipresent,Omniscient
0,Phantom,Male,-,-,-,-99.0,DC Comics,-,good,-99.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
1,Absorbing Man,Male,blue,Human,No Hair,193.0,Marvel Comics,-,bad,122.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
2,Deadshot,Male,brown,Human,Brown,185.0,DC Comics,-,bad,91.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
3,Hal Jordan,Male,brown,Human,Brown,188.0,DC Comics,-,good,90.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
4,Black Flash,Male,-,God / Eternal,-,-99.0,DC Comics,-,neutral,-99.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
5,Zatanna,Female,blue,Human,Black,170.0,DC Comics,-,good,57.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
6,Captain America,Male,blue,Human,blond,188.0,Marvel Comics,-,good,108.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
7,Captain Epic,Male,blue,-,Brown,188.0,Team Epic TV,-,good,-99.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
8,Vulture,Male,brown,Human,No Hair,180.0,Marvel Comics,-,bad,79.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
9,Hyperion,Male,blue,Eternal,Red,183.0,Marvel Comics,-,good,207.0,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [5]:
with open_file(os.path.join(MODEL_DIR,"trainset.csv"), "w") as f:
    df.to_csv(f,index=False)
with open_file(os.path.join(MODEL_DIR,"evalset.csv"), "w") as f:
    df_eval.to_csv(f,index=False)

In [6]:
exclude_columns=["Publisher","Height","Weight","Skin_color","Race"]
INDEX=df.columns[0]
COLUMNS=[x for x in df.columns[1:] if x not in exclude_columns]
LABEL_FIELD="Alignment"

In [7]:
FIELD_DEFAULTS=[]
FIELD_TYPES={}
FIELD_CATEGORIES={}
dtypes=dict(df.dtypes)
for c in COLUMNS:
    if(str(dtypes[c])=="bool"):
        FIELD_DEFAULTS.append([0])
        FIELD_TYPES[c]="bool"
    elif(str(dtypes[c])=="object"):
        FIELD_DEFAULTS.append(["NA"])
        FIELD_TYPES[c]="string"
        FIELD_CATEGORIES[c]=list(sorted(set(list(df[c].unique())+["NA"])))
    else:  
        FIELD_DEFAULTS.append([0.0])
        FIELD_TYPES[c]="number"
FIELD_CATEGORIES[LABEL_FIELD]=[x for x in FIELD_CATEGORIES[LABEL_FIELD] if x != "NA"]

In [8]:
with open_file(os.path.join(MODEL_DIR,"data","dataset_fields.json"), "w") as f:
    json.dump(
        {
            "fields":{
                "columns" : COLUMNS,#[x for x in COLUMNS if x != LABEL_FIELD],
                "types" : FIELD_TYPES,#{x:FIELD_TYPES[x] for x in FIELD_TYPES if x != LABEL_FIELD},
                "categories" : FIELD_CATEGORIES,#{x:FIELD_CATEGORIES[x] for x in FIELD_CATEGORIES if x != LABEL_FIELD}
            },
            "label":{
                "column" : LABEL_FIELD,
                "type" : FIELD_TYPES[LABEL_FIELD],
                "categories" : FIELD_CATEGORIES[LABEL_FIELD],
            }
        },
        f
    )

In [9]:
with open_file(os.path.join(MODEL_DIR,"data","tf_trainset.csv"), "w") as f:
    df[COLUMNS].to_csv(f, index=False)
with open_file(os.path.join(MODEL_DIR,"data","tf_evalset.csv"), "w") as f:
    df_eval[COLUMNS].to_csv(f, index=False)

## hyperparam.yaml

In [10]:
config="""trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxTrials: 50
    maxParallelTrials: 5
    params:
    - parameterName: LEARNING_RATE
      type: DOUBLE
      minValue: 0.00001
      maxValue: 0.01
      scaleType: UNIT_LOG_SCALE
    - parameterName: BATCH_SIZE
      type: INTEGER
      minValue: 16
      maxValue: 1024
      scaleType: UNIT_LOG_SCALE
"""

with open_file(os.path.join(MODEL_DIR,"config","hyperparam.yaml"), "w") as f:
    df.to_csv(f,index=False)

print(config)
with open("hyperparam.yaml", "w") as f:
    f.write(config)
    
with open("hyperparam.yaml", "r") as f:
    print(f.read())

trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxTrials: 50
    maxParallelTrials: 5
    params:
    - parameterName: LEARNING_RATE
      type: DOUBLE
      minValue: 0.00001
      maxValue: 0.01
      scaleType: UNIT_LOG_SCALE
    - parameterName: BATCH_SIZE
      type: INTEGER
      minValue: 16
      maxValue: 1024
      scaleType: UNIT_LOG_SCALE

trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxTrials: 50
    maxParallelTrials: 5
    params:
    - parameterName: LEARNING_RATE
      type: DOUBLE
      minValue: 0.00001
      maxValue: 0.01
      scaleType: UNIT_LOG_SCALE
    - parameterName: BATCH_SIZE
      type: INTEGER
      minValue: 16
      maxValue: 1024
      scaleType: UNIT_LOG_SCALE



In [ ]:
config="""trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 50
    maxParallelTrials: 5
    params:
    - parameterName: LEARNING_RATE
      type: DOUBLE
      minValue: 0.001
      maxValue: 0.01
      scaleType: UNIT_LOG_SCALE
    - parameterName: BATCH_SIZE
      type: INTEGER
      minValue: 16
      maxValue: 32
      scaleType: UNIT_LOG_SCALE
"""
with open_file(os.path.join(MODEL_DIR,"config","hyperparam.yaml"), "w") as f:
    df.to_csv(f,index=False)

print(config)
with open("hyperparam.yaml", "w") as f:
    f.write(config)
    
with open("hyperparam.yaml", "r") as f:
    print(f.read())

# Train

In [11]:
os.environ['MODEL_DIR'] = str(MODEL_DIR)
os.environ['TRAIN_STEPS'] = str(TRAIN_STEPS)
os.environ['BATCH_SIZE'] = str(BATCH_SIZE)
os.environ['LEARNING_RATE'] = str(LEARNING_RATE)

os.environ['BUCKET'] = "ml-research-injenia"
os.environ['REGION'] = 'europe-west1'

## Local

In [ ]:
%%bash

python -u trainer/task.py \
    --MODEL_DIR=$MODEL_DIR \
    --TRAIN_STEPS $TRAIN_STEPS \
    --BATCH_SIZE $BATCH_SIZE   \
    --LEARNING_RATE $LEARNING_RATE 

## Cloud

In [12]:
%%bash

JOBNAME=superheroes_$(date -u +%y%m%d_%H%M%S)

echo "Launching training job ... trained model will be in $MODEL_DIR"
#gsutil -m rm -rf $OUTPUT_DIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$(pwd)/trainer \
  --job-dir=$MODEL_DIR \
  --staging-bucket=gs://$BUCKET-staging \
  --runtime-version="1.6" \
  --config=hyperparam.yaml \
  -- \
   --MODEL_DIR=$MODEL_DIR \
   --TRAIN_STEPS $TRAIN_STEPS #\
#   --BATCH_SIZE $BATCH_SIZE   \
#   --LEARNING_RATE $LEARNING_RATE 

Launching training job ... trained model will be in gs://ml-research-injenia/estimators/trainings-superheroes-v2/linear-classifier/test06
jobId: superheroes_180906_151240
state: QUEUED


Job [superheroes_180906_151240] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe superheroes_180906_151240

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs superheroes_180906_151240


In [3]:
import subprocess
import json
import pandas as pd


def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z


res_str=subprocess.check_output("gcloud ml-engine jobs describe superheroes_180906_151240 --format json".split(" "))
res=json.loads(res_str)
trials=res["trainingOutput"]["trials"]
df=pd.DataFrame([
    merge_two_dicts({"trialId":t["trialId"]},
    merge_two_dicts(t["finalMetric"],t["hyperparameters"]) )
    for t in trials])

df

,BATCH_SIZE,LEARNING_RATE,objectiveValue,trainingStep,trialId
0,366,0.000769597311241036,0.750055,100008,45
1,959,0.00038898791218559072,0.750031,100006,40
2,1009,0.00049508362445768435,0.750010,100008,26
3,961,0.00046727261211331221,0.750000,100007,1
4,16,0.0079510771090190446,0.750000,100007,3
5,16,0.0078899095417919,0.750000,100008,16
6,16,0.0079669890618274137,0.750000,100007,17
7,1024,0.00054652602888491321,0.750000,100007,21
8,1023,0.00036933021665258228,0.750000,100007,23
9,1024,0.00050975432522034822,0.750000,100008,29
